In [1]:
import sys
sys.path.append('../')

from groof.config import *

from pathlib import Path

from groof.geo import *
import geopandas as gpd
from shapely.geometry import box
import rioxarray as rio
from sklearn.model_selection import train_test_split

In [2]:

bbox = gpd.read_file(BBOX)
utm_crs = bbox.estimate_utm_crs()
bbox=bbox.to_crs(utm_crs)

## Reproject To Local UTM CRS

In [3]:
# reproject_rio(src_fp=S2_MEDIAN, dst_fp=S2_MOSAIC_REPROJ, dst_crs=utm_crs)


# reproject_rio(src_fp=BU_MEDIAN, dst_fp=bu_reproj_fp, dst_crs=utm_crs)
# reproject_rio(src_fp=NDWI_MEDIAN, dst_fp=ndwi_reproj_fp, dst_crs=utm_crs)
# reproject_rio(src_fp=EVI_MEDIAN, dst_fp=evi_reproj_fp, dst_crs=utm_crs)

## Rasterize Buildings

In [4]:
ref = rio.open_rasterio(EVI_MEDIAN)
bldg = gpd.read_file(BLDG_FP).to_crs(utm_crs)
bldg_clip = bldg.clip(bbox)[['HEIGHT', 'geometry']]
bldg_clip['class'] = 1.0

# Building Heights Raster
rasterize_vector(
    gdf= bldg_clip,
    ref= ref,
    attribute = 'HEIGHT',
    out_path = PRC / 'building_height.tif'
)

# Building Binary Mask
rasterize_vector(
    gdf= bldg_clip,
    ref= ref,
    attribute = 'class',
    out_path = PRC / 'building_mask.tif',
    # fill_value = 0
)

## Prepare Training Data

In [5]:
groof_train = gpd.read_file(GROOF_TRAIN_AREAS).to_crs(utm_crs)
groof_train['class'] = [2.0 if cl == 'green_roof' else 1.0 for cl in groof_train.class_name]
# groof_train = groof_train[groof_train.class_name == 'green_roof']
# groof_train['class'] = 1.0
bldg_groof = bldg_clip.sjoin(groof_train.drop(columns=['class'])).drop(columns=['index_right']).drop_duplicates('geometry')


rasterize_vector(
    gdf= bldg_groof,
    ref= ref,
    attribute = 'class',
    out_path = PRC / 'bldg_groof.tif',
    # fill_value = 0
)

rasterize_vector(groof_train, 
                 ref=ref, 
                 attribute='class',
                 crs = utm_crs,
                 out_path = PRC / 'groof.tif'

)

## Create Ground Truth Raster

In [6]:
bg = rio.open_rasterio(PRC / 'bldg_groof.tif')
gr = rio.open_rasterio(PRC / 'groof.tif')

groundtruth = bg.where(gr != 2, gr)
groundtruth.rio.to_raster(PRC / 'bldg_groof_groundtruth.tif')




In [9]:
train, test = train_test_split(bldg_groof, test_size=0.2, random_state=42)


train = groundtruth.rio.clip(train.geometry.values).rio.reproject_match(ref)
train.rio.to_raster(TRAIN)


test = groundtruth.rio.clip(test.geometry.values).rio.reproject_match(ref)
test.rio.to_raster(TEST)

## Match Rasters to Sentinel-2

In [10]:
# groof_train = gpd.read_file(GROOF_TRAIN_AREAS).to_crs(utm_crs)
# groof_train['class'] = [2.0 if cl == 'green_roof' else 1.0 for cl in groof_train.class_name]
# train, test = train_test_split(groof_train, test_size=0.2, stratify=groof_train['class'], random_state=42)


# # rasterize_vector(test, 
# #                  ref=ref, 
# #                  attribute='class',
# #                  crs = utm_crs,
# #                  fill_value = 0,
# #                  out_path = 'data/prc/test.tif'
# # )

# rasterize_vector(train, 
#                  ref=ref, 
#                  attribute='class',
#                  crs = utm_crs,
#                  fill_value = 0,
#                  out_path = TRAIN
# )


# rasterize_vector(test, 
#                  ref=ref, 
#                  attribute='class',
#                  crs = utm_crs,
#                  fill_value = 0,
#                  out_path = TEST
# )
